In [1]:
import pandas as pd
from dataset_functions import Dataset_Formatter
from mordred import Calculator, descriptors
from glob import glob

mk = Dataset_Formatter()
 

In [51]:
files = glob('/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_*.csv.gz')
files

['/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_rdkit_3.csv.gz',
 '/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_rdkit_4.csv.gz',
 '/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_rdkit_2.csv.gz',
 '/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_rdkit_1.csv.gz']

In [52]:
import re
# Function to extract the number from the file name
def extract_number(file_path):
    match = re.search(r'PMG_rdkit_(\d+)', file_path)
    return int(match.group(1)) if match else float('inf')

# Sort the list using the extracted numbers
sorted_file_paths = sorted(files, key=extract_number)
print(sorted_file_paths)


['/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_rdkit_1.csv.gz', '/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_rdkit_2.csv.gz', '/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_rdkit_3.csv.gz', '/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_rdkit_4.csv.gz']


In [57]:
for file in sorted_file_paths:
    print(file)
    chunks = pd.read_csv(file, compression='gzip', chunksize=200000)
    for chunk in chunks:
        print(chunk)

/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_rdkit_1.csv.gz


KeyboardInterrupt: 

In [73]:
test_desc = pd.read_csv('/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/PMG_rdkit_desc_31.csv', index_col='ID').iloc[:-3000]
test_desc.to_csv('/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/PMG_rdkit_desc_31.csv', index_label='ID')

In [74]:
test_dock = pd.read_csv('~/Recreating_DMTA/datasets/PyMolGen/docking/PMG_docking_31.csv', index_col='ID').iloc[:-3000]
test_dock.to_csv('~/Recreating_DMTA/datasets/PyMolGen/docking/PMG_docking_31.csv', index_label='ID')

In [77]:
test_full = pd.read_csv('/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_rdkit_31.csv', index_col='ID').iloc[:-3000]
test_full.to_csv('/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_rdkit_31.csv', index_label='ID')

In [76]:
test_full = pd.read_csv('/users/yhb18174/Recreating_DMTA/datasets/PyMolGen/desc/rdkit/full_data/PMG_rdkit_31.csv', index_col='ID').tail(3000)
test_full.to_csv('/users/yhb18174/Recreating_DMTA/datasets/held_out_data/PMG_rdkit_full.csv', index_label='ID')
